In [1]:
pip install librosa praat-parselmouth numpy scipy nolds

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import librosa
import parselmouth
import numpy
import scipy
import nolds

print("All libraries imported successfully!")

All libraries imported successfully!


In [3]:
import librosa
import parselmouth
import numpy as np
from scipy.signal import find_peaks
import nolds

# Load audio file
def load_audio(file_path):
    y, sr = librosa.load(file_path, sr=None)  # Load with native sample rate
    return y, sr

# Extract pitch and amplitude contours
def extract_pitch_and_amplitude(file_path, y, sr):
    # Using Parselmouth for pitch (more precise than librosa for voice analysis)
    snd = parselmouth.Sound(file_path)
    pitch = snd.to_pitch(time_step=0.01, pitch_floor=75, pitch_ceiling=600)  # Typical voice range
    pitch_values = pitch.selected_array['frequency']
    pitch_values[pitch_values == 0] = np.nan  # Unvoiced segments as NaN
    pitch_times = pitch.xs()

    # Amplitude contour (RMS energy)
    frame_length = int(0.03 * sr)  # 30ms frame
    hop_length = int(0.01 * sr)   # 10ms hop
    rms = librosa.feature.rms(y=y, frame_length=frame_length, hop_length=hop_length)[0]
    rms_times = librosa.times_like(rms, sr=sr, hop_length=hop_length)

    return pitch_values, pitch_times, rms, rms_times

# Compute jitter variants
def compute_jitter(pitch_values):
    pitch_periods = 1 / pitch_values[~np.isnan(pitch_values)]  # Convert Hz to seconds
    if len(pitch_periods) < 5:  # Need enough periods for meaningful stats
        return np.nan, np.nan, np.nan, np.nan, np.nan
    
    # Jitter (absolute) in seconds
    jitter_abs = np.mean(np.abs(np.diff(pitch_periods)))
    # Jitter (%) relative to mean period
    jitter_percent = jitter_abs / np.mean(pitch_periods) * 100
    # RAP (Relative Average Perturbation, 3-frame)
    rap = np.mean([np.abs(pitch_periods[i] - np.mean(pitch_periods[i-1:i+2])) 
                   for i in range(1, len(pitch_periods)-1)]) / np.mean(pitch_periods)
    # PPQ (Period Perturbation Quotient, 5-frame)
    ppq = np.mean([np.abs(pitch_periods[i] - np.mean(pitch_periods[max(0, i-2):min(len(pitch_periods), i+3)])) 
                   for i in range(2, len(pitch_periods)-2)]) / np.mean(pitch_periods)
    # DDP (Difference of Differences of Periods, 3x RAP)
    ddp = 3 * rap
    
    return jitter_percent, jitter_abs, rap, ppq, ddp

# Compute shimmer variants
def compute_shimmer(y, sr):
    # Find amplitude peaks (approximate glottal pulses)
    peaks, _ = find_peaks(np.abs(y), distance=int(sr/500), height=np.mean(np.abs(y))*0.1)
    amplitudes = np.abs(y[peaks])
    if len(amplitudes) < 5:
        return np.nan, np.nan, np.nan, np.nan, np.nan
    
    # Shimmer (relative amplitude variation)
    shimmer = np.mean(np.abs(np.diff(amplitudes))) / np.mean(amplitudes)
    # Shimmer (dB)
    shimmer_db = 20 * np.log10(1 + shimmer)
    # APQ3 (3-frame Amplitude Perturbation Quotient)
    apq3 = np.mean([np.abs(amplitudes[i] - np.mean(amplitudes[max(0, i-1):min(len(amplitudes), i+2)])) 
                    for i in range(1, len(amplitudes)-1)]) / np.mean(amplitudes)
    # APQ5 (5-frame Amplitude Perturbation Quotient)
    apq5 = np.mean([np.abs(amplitudes[i] - np.mean(amplitudes[max(0, i-2):min(len(amplitudes), i+3)])) 
                    for i in range(2, len(amplitudes)-2)]) / np.mean(amplitudes)
    # DDA (Difference of Differences of Amplitudes, 3x APQ3)
    dda = 3 * apq3
    
    return shimmer, shimmer_db, apq3, apq5, dda

# Compute HNR and NHR
def compute_hnr_nhr(file_path):
    snd = parselmouth.Sound(file_path)
    # Corrected argument: use 'minimum_pitch' instead of 'minimum_frequency'
    harmonicity = snd.to_harmonicity(time_step=0.01, minimum_pitch=75)
    hnr = np.nanmean(harmonicity.values[harmonicity.values > -200])  # Valid HNR values only
    nhr = 10**(-hnr/10) if hnr is not np.nan else np.nan  # NHR = 1 / (10^(HNR/10))
    return hnr, nhr

# Compute nonlinear features (RPDE, DFA, spread1, spread2, D2, PPE)
def compute_nonlinear_features(pitch_values):
    pitch_clean = pitch_values[~np.isnan(pitch_values)]
    if len(pitch_clean) < 50:  # Need sufficient data points
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan
    
    # DFA (Detrended Fluctuation Analysis)
    dfa = nolds.dfa(pitch_clean)
    
    # RPDE (Recurrence Period Density Entropy) - Simplified approximation
    periods = 1 / pitch_clean
    period_diffs = np.diff(periods)
    hist, bins = np.histogram(period_diffs, bins=20, density=True)
    hist = hist / np.sum(hist)  # Normalize to probability
    rpde = -np.sum(hist * np.log2(hist + 1e-10))  # Entropy (avoid log(0))
    
    # Spread1 and Spread2 (Poincaré plot measures)
    diffs = np.diff(pitch_clean)
    sd1 = np.std(diffs)  # Short-term variability (spread1 approximation)
    sd2 = np.std(pitch_clean)  # Long-term variability (spread2 approximation)
    
    # D2 (Correlation Dimension) - Very simplified approximation
    d2 = nolds.corr_dim(pitch_clean, emb_dim=2)  # Embedding dimension 2
    
    # PPE (Pitch Period Entropy) - Entropy of pitch periods
    pitch_hist, _ = np.histogram(pitch_clean, bins=20, density=True)
    pitch_hist = pitch_hist / np.sum(pitch_hist)
    ppe = -np.sum(pitch_hist * np.log2(pitch_hist + 1e-10))
    
    return rpde, dfa, sd1, sd2, d2, ppe

# Main function to extract all features
def extract_voice_features(file_path):
    # Load audio
    y, sr = load_audio(file_path)
    
    # Extract pitch and amplitude
    pitch_values, pitch_times, rms, rms_times = extract_pitch_and_amplitude(file_path, y, sr)
    
    # Fundamental frequency features
    fo = np.nanmean(pitch_values)  # Mean pitch
    fhi = np.nanmax(pitch_values)  # Max pitch
    flo = np.nanmin(pitch_values)  # Min pitch
    
    # Jitter features
    jitter_percent, jitter_abs, rap, ppq, ddp = compute_jitter(pitch_values)
    
    # Shimmer features
    shimmer, shimmer_db, apq3, apq5, dda = compute_shimmer(y, sr)
    
    # HNR and NHR
    hnr, nhr = compute_hnr_nhr(file_path)
    
    # Nonlinear features
    rpde, dfa, spread1, spread2, d2, ppe = compute_nonlinear_features(pitch_values)
    
    # Compile results into a dictionary
    features = {
        "MDVP:Fo(Hz)": fo,
        "MDVP:Fhi(Hz)": fhi,
        "MDVP:Flo(Hz)": flo,
        "MDVP:Jitter(%)": jitter_percent,
        "MDVP:Jitter(Abs)": jitter_abs,
        "MDVP:RAP": rap,
        "MDVP:PPQ": ppq,
        "Jitter:DDP": ddp,
        "MDVP:Shimmer": shimmer,
        "MDVP:Shimmer(dB)": shimmer_db,
        "Shimmer:APQ3": apq3,
        "Shimmer:APQ5": apq5,
        "MDVP:APQ": apq5,  # Approximation (MDVP uses 11 frames, here we use 5)
        "Shimmer:DDA": dda,
        "NHR": nhr,
        "HNR": hnr,
        "RPDE": rpde,
        "DFA": dfa,
        "spread1": spread1,
        "spread2": spread2,
        "D2": d2,
        "PPE": ppe,
        "status": None  # Not extractable from audio; requires external label
    }
    
    return features

# Example usage
if __name__ == "__main__":
    file_path = r"C:\Users\sures\OneDrive\Desktop\parkinsons-code\voice_files_wav\Recording.wav"  # Your .wav file path
    try:
        features = extract_voice_features(file_path)
        print("Extracted Voice Features:")
        for key, value in features.items():
            print(f"{key}: {value:.6f}" if isinstance(value, (float, np.float64)) else f"{key}: {value}")
    except Exception as e:
        print(f"Error processing file: {e}")

Extracted Voice Features:
MDVP:Fo(Hz): 146.381432
MDVP:Fhi(Hz): 470.980002
MDVP:Flo(Hz): 81.886046
MDVP:Jitter(%): 2.926188
MDVP:Jitter(Abs): 0.000209
MDVP:RAP: 0.013754
MDVP:PPQ: 0.022949
Jitter:DDP: 0.041262
MDVP:Shimmer: 0.6933469176292419
MDVP:Shimmer(dB): 4.574919
Shimmer:APQ3: 0.4218675196170807
Shimmer:APQ5: 0.39511770009994507
MDVP:APQ: 0.39511770009994507
Shimmer:DDA: 1.265603
NHR: 0.128314
HNR: 8.917244
RPDE: 0.902523
DFA: 1.023307
spread1: 17.698635
spread2: 34.360202
D2: 1.243999
PPE: 2.607913
status: None


In [4]:
convert it to json
give the input json data to the model

SyntaxError: invalid syntax (2855735588.py, line 1)

In [1]:
import librosa
import parselmouth
import numpy as np
from scipy.signal import find_peaks
import nolds
import json

# Load audio file
def load_audio(file_path):
    y, sr = librosa.load(file_path, sr=None)  # Load with native sample rate
    return y, sr

# Extract pitch and amplitude contours
def extract_pitch_and_amplitude(file_path, y, sr):
    snd = parselmouth.Sound(file_path)
    pitch = snd.to_pitch(time_step=0.01, pitch_floor=75, pitch_ceiling=600)  # Typical voice range
    pitch_values = pitch.selected_array['frequency']
    pitch_values[pitch_values == 0] = np.nan  # Unvoiced segments as NaN
    pitch_times = pitch.xs()

    frame_length = int(0.03 * sr)  # 30ms frame
    hop_length = int(0.01 * sr)   # 10ms hop
    rms = librosa.feature.rms(y=y, frame_length=frame_length, hop_length=hop_length)[0]
    rms_times = librosa.times_like(rms, sr=sr, hop_length=hop_length)

    return pitch_values, pitch_times, rms, rms_times

# Compute jitter variants
def compute_jitter(pitch_values):
    pitch_periods = 1 / pitch_values[~np.isnan(pitch_values)]  # Convert Hz to seconds
    if len(pitch_periods) < 5:  # Need enough periods for meaningful stats
        return np.nan, np.nan, np.nan, np.nan, np.nan
    
    jitter_abs = np.mean(np.abs(np.diff(pitch_periods)))
    jitter_percent = jitter_abs / np.mean(pitch_periods) * 100
    rap = np.mean([np.abs(pitch_periods[i] - np.mean(pitch_periods[i-1:i+2])) 
                   for i in range(1, len(pitch_periods)-1)]) / np.mean(pitch_periods)
    ppq = np.mean([np.abs(pitch_periods[i] - np.mean(pitch_periods[max(0, i-2):min(len(pitch_periods), i+3)])) 
                   for i in range(2, len(pitch_periods)-2)]) / np.mean(pitch_periods)
    ddp = 3 * rap
    
    return jitter_percent, jitter_abs, rap, ppq, ddp

# Compute shimmer variants
def compute_shimmer(y, sr):
    peaks, _ = find_peaks(np.abs(y), distance=int(sr/500), height=np.mean(np.abs(y))*0.1)
    amplitudes = np.abs(y[peaks])
    if len(amplitudes) < 5:
        return np.nan, np.nan, np.nan, np.nan, np.nan
    
    shimmer = np.mean(np.abs(np.diff(amplitudes))) / np.mean(amplitudes)
    shimmer_db = 20 * np.log10(1 + shimmer)
    apq3 = np.mean([np.abs(amplitudes[i] - np.mean(amplitudes[max(0, i-1):min(len(amplitudes), i+2)])) 
                    for i in range(1, len(amplitudes)-1)]) / np.mean(amplitudes)
    apq5 = np.mean([np.abs(amplitudes[i] - np.mean(amplitudes[max(0, i-2):min(len(amplitudes), i+3)])) 
                    for i in range(2, len(amplitudes)-2)]) / np.mean(amplitudes)
    dda = 3 * apq3
    
    return shimmer, shimmer_db, apq3, apq5, dda

# Compute HNR and NHR
def compute_hnr_nhr(file_path):
    snd = parselmouth.Sound(file_path)
    harmonicity = snd.to_harmonicity(time_step=0.01, minimum_pitch=75)
    hnr = np.nanmean(harmonicity.values[harmonicity.values > -200])  # Valid HNR values only
    nhr = 10**(-hnr/10) if hnr is not np.nan else np.nan  # NHR = 1 / (10^(HNR/10))
    return hnr, nhr

# Compute nonlinear features (RPDE, DFA, spread1, spread2, D2, PPE)
def compute_nonlinear_features(pitch_values):
    pitch_clean = pitch_values[~np.isnan(pitch_values)]
    if len(pitch_clean) < 50:  # Need sufficient data points
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan
    
    dfa = nolds.dfa(pitch_clean)
    periods = 1 / pitch_clean
    period_diffs = np.diff(periods)
    hist, bins = np.histogram(period_diffs, bins=20, density=True)
    hist = hist / np.sum(hist)  # Normalize to probability
    rpde = -np.sum(hist * np.log2(hist + 1e-10))  # Entropy (avoid log(0))
    
    diffs = np.diff(pitch_clean)
    sd1 = np.std(diffs)  # Short-term variability (spread1 approximation)
    sd2 = np.std(pitch_clean)  # Long-term variability (spread2 approximation)
    
    d2 = nolds.corr_dim(pitch_clean, emb_dim=2)  # Embedding dimension 2
    
    pitch_hist, _ = np.histogram(pitch_clean, bins=20, density=True)
    pitch_hist = pitch_hist / np.sum(pitch_hist)
    ppe = -np.sum(pitch_hist * np.log2(pitch_hist + 1e-10))
    
    return rpde, dfa, sd1, sd2, d2, ppe

# Main function to extract all features
def extract_voice_features(file_path):
    y, sr = load_audio(file_path)
    pitch_values, pitch_times, rms, rms_times = extract_pitch_and_amplitude(file_path, y, sr)
    
    fo = np.nanmean(pitch_values)
    fhi = np.nanmax(pitch_values)
    flo = np.nanmin(pitch_values)
    
    jitter_percent, jitter_abs, rap, ppq, ddp = compute_jitter(pitch_values)
    shimmer, shimmer_db, apq3, apq5, dda = compute_shimmer(y, sr)
    hnr, nhr = compute_hnr_nhr(file_path)
    rpde, dfa, spread1, spread2, d2, ppe = compute_nonlinear_features(pitch_values)
    
    features = {
        "MDVP:Fo(Hz)": float(fo) if not np.isnan(fo) else None,
        "MDVP:Fhi(Hz)": float(fhi) if not np.isnan(fhi) else None,
        "MDVP:Flo(Hz)": float(flo) if not np.isnan(flo) else None,
        "MDVP:Jitter(%)": float(jitter_percent) if not np.isnan(jitter_percent) else None,
        "MDVP:Jitter(Abs)": float(jitter_abs) if not np.isnan(jitter_abs) else None,
        "MDVP:RAP": float(rap) if not np.isnan(rap) else None,
        "MDVP:PPQ": float(ppq) if not np.isnan(ppq) else None,
        "Jitter:DDP": float(ddp) if not np.isnan(ddp) else None,
        "MDVP:Shimmer": float(shimmer) if not np.isnan(shimmer) else None,
        "MDVP:Shimmer(dB)": float(shimmer_db) if not np.isnan(shimmer_db) else None,
        "Shimmer:APQ3": float(apq3) if not np.isnan(apq3) else None,
        "Shimmer:APQ5": float(apq5) if not np.isnan(apq5) else None,
        "MDVP:APQ": float(apq5) if not np.isnan(apq5) else None,  # Approximation
        "Shimmer:DDA": float(dda) if not np.isnan(dda) else None,
        "NHR": float(nhr) if not np.isnan(nhr) else None,
        "HNR": float(hnr) if not np.isnan(hnr) else None,
        "RPDE": float(rpde) if not np.isnan(rpde) else None,
        "DFA": float(dfa) if not np.isnan(dfa) else None,
        "spread1": float(spread1) if not np.isnan(spread1) else None,
        "spread2": float(spread2) if not np.isnan(spread2) else None,
        "D2": float(d2) if not np.isnan(d2) else None,
        "PPE": float(ppe) if not np.isnan(ppe) else None,
        "status": None  # Not extractable from audio
    }
    
    return features

# Function to save features to JSON
def save_features_to_json(features, output_file):
    with open(output_file, 'w') as f:
        json.dump(features, f, indent=4)
    print(f"Features saved to {output_file}")

# Example usage
if __name__ == "__main__":
    file_path = r"C:\Users\sures\OneDrive\Desktop\parkinsons-code\PD_AH\AH_545616858-3A749CBC-3FEB-4D35-820E-E45C3E5B9B6A.wav"
    output_file = r"C:\Users\sures\OneDrive\Desktop\parkinsons-code\features4.json"  # Output JSON file path
    
    try:
        features = extract_voice_features(file_path)
        print("Extracted Voice Features:")
        for key, value in features.items():
            print(f"{key}: {value:.6f}" if isinstance(value, (float, np.float64)) else f"{key}: {value}")
        
        # Save to JSON
        save_features_to_json(features, output_file)
    except Exception as e:
        print(f"Error processing file: {e}")

Extracted Voice Features:
MDVP:Fo(Hz): 126.353888
MDVP:Fhi(Hz): 137.019936
MDVP:Flo(Hz): 120.319498
MDVP:Jitter(%): 0.359802
MDVP:Jitter(Abs): 0.000028
MDVP:RAP: 0.001296
MDVP:PPQ: 0.001894
Jitter:DDP: 0.003889
MDVP:Shimmer: 0.468201
MDVP:Shimmer(dB): 3.335713
Shimmer:APQ3: 0.258954
Shimmer:APQ5: 0.305873
MDVP:APQ: 0.305873
Shimmer:DDA: 0.776862
NHR: 0.043201
HNR: 13.645093
RPDE: 3.806459
DFA: 1.343438
spread1: 0.566512
spread2: 3.372102
D2: 1.478352
PPE: 3.673118
status: None
Features saved to C:\Users\sures\OneDrive\Desktop\parkinsons-code\features4.json


In [2]:
import librosa
import parselmouth
import numpy as np
from scipy.signal import find_peaks
import nolds
import json
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import joblib

# --- Voice Feature Extraction Functions ---

def load_audio(file_path):
    y, sr = librosa.load(file_path, sr=None)
    return y, sr

def extract_pitch_and_amplitude(file_path, y, sr):
    snd = parselmouth.Sound(file_path)
    pitch = snd.to_pitch(time_step=0.01, pitch_floor=75, pitch_ceiling=600)
    pitch_values = pitch.selected_array['frequency']
    pitch_values[pitch_values == 0] = np.nan
    pitch_times = pitch.xs()

    frame_length = int(0.03 * sr)
    hop_length = int(0.01 * sr)
    rms = librosa.feature.rms(y=y, frame_length=frame_length, hop_length=hop_length)[0]
    rms_times = librosa.times_like(rms, sr=sr, hop_length=hop_length)
    return pitch_values, pitch_times, rms, rms_times

def compute_jitter(pitch_values):
    pitch_periods = 1 / pitch_values[~np.isnan(pitch_values)]
    if len(pitch_periods) < 5:
        return np.nan, np.nan, np.nan, np.nan, np.nan
    
    jitter_abs = np.mean(np.abs(np.diff(pitch_periods)))
    jitter_percent = jitter_abs / np.mean(pitch_periods) * 100
    rap = np.mean([np.abs(pitch_periods[i] - np.mean(pitch_periods[i-1:i+2])) 
                   for i in range(1, len(pitch_periods)-1)]) / np.mean(pitch_periods)
    ppq = np.mean([np.abs(pitch_periods[i] - np.mean(pitch_periods[max(0, i-2):min(len(pitch_periods), i+3)])) 
                   for i in range(2, len(pitch_periods)-2)]) / np.mean(pitch_periods)
    ddp = 3 * rap
    return jitter_percent, jitter_abs, rap, ppq, ddp

def compute_shimmer(y, sr):
    peaks, _ = find_peaks(np.abs(y), distance=int(sr/500), height=np.mean(np.abs(y))*0.1)
    amplitudes = np.abs(y[peaks])
    if len(amplitudes) < 5:
        return np.nan, np.nan, np.nan, np.nan, np.nan
    
    shimmer = np.mean(np.abs(np.diff(amplitudes))) / np.mean(amplitudes)
    shimmer_db = 20 * np.log10(1 + shimmer)
    apq3 = np.mean([np.abs(amplitudes[i] - np.mean(amplitudes[max(0, i-1):min(len(amplitudes), i+2)])) 
                    for i in range(1, len(amplitudes)-1)]) / np.mean(amplitudes)
    apq5 = np.mean([np.abs(amplitudes[i] - np.mean(amplitudes[max(0, i-2):min(len(amplitudes), i+3)])) 
                    for i in range(2, len(amplitudes)-2)]) / np.mean(amplitudes)
    dda = 3 * apq3
    return shimmer, shimmer_db, apq3, apq5, dda

def compute_hnr_nhr(file_path):
    snd = parselmouth.Sound(file_path)
    harmonicity = snd.to_harmonicity(time_step=0.01, minimum_pitch=75)
    hnr = np.nanmean(harmonicity.values[harmonicity.values > -200])
    nhr = 10**(-hnr/10) if hnr is not np.nan else np.nan
    return hnr, nhr

def compute_nonlinear_features(pitch_values):
    pitch_clean = pitch_values[~np.isnan(pitch_values)]
    if len(pitch_clean) < 50:
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan
    
    dfa = nolds.dfa(pitch_clean)
    periods = 1 / pitch_clean
    period_diffs = np.diff(periods)
    hist, bins = np.histogram(period_diffs, bins=20, density=True)
    hist = hist / np.sum(hist)
    rpde = -np.sum(hist * np.log2(hist + 1e-10))
    
    diffs = np.diff(pitch_clean)
    sd1 = np.std(diffs)
    sd2 = np.std(pitch_clean)
    
    d2 = nolds.corr_dim(pitch_clean, emb_dim=2)
    
    pitch_hist, _ = np.histogram(pitch_clean, bins=20, density=True)
    pitch_hist = pitch_hist / np.sum(pitch_hist)
    ppe = -np.sum(pitch_hist * np.log2(pitch_hist + 1e-10))
    return rpde, dfa, sd1, sd2, d2, ppe

def extract_voice_features(file_path):
    y, sr = load_audio(file_path)
    pitch_values, pitch_times, rms, rms_times = extract_pitch_and_amplitude(file_path, y, sr)
    
    fo = np.nanmean(pitch_values)
    fhi = np.nanmax(pitch_values)
    flo = np.nanmin(pitch_values)
    
    jitter_percent, jitter_abs, rap, ppq, ddp = compute_jitter(pitch_values)
    shimmer, shimmer_db, apq3, apq5, dda = compute_shimmer(y, sr)
    hnr, nhr = compute_hnr_nhr(file_path)
    rpde, dfa, spread1, spread2, d2, ppe = compute_nonlinear_features(pitch_values)
    
    features = {
        "MDVP:Fo(Hz)": float(fo) if not np.isnan(fo) else None,
        "MDVP:Fhi(Hz)": float(fhi) if not np.isnan(fhi) else None,
        "MDVP:Flo(Hz)": float(flo) if not np.isnan(flo) else None,
        "MDVP:Jitter(%)": float(jitter_percent) if not np.isnan(jitter_percent) else None,
        "MDVP:Jitter(Abs)": float(jitter_abs) if not np.isnan(jitter_abs) else None,
        "MDVP:RAP": float(rap) if not np.isnan(rap) else None,
        "MDVP:PPQ": float(ppq) if not np.isnan(ppq) else None,
        "Jitter:DDP": float(ddp) if not np.isnan(ddp) else None,
        "MDVP:Shimmer": float(shimmer) if not np.isnan(shimmer) else None,
        "MDVP:Shimmer(dB)": float(shimmer_db) if not np.isnan(shimmer_db) else None,
        "Shimmer:APQ3": float(apq3) if not np.isnan(apq3) else None,
        "Shimmer:APQ5": float(apq5) if not np.isnan(apq5) else None,
        "MDVP:APQ": float(apq5) if not np.isnan(apq5) else None,
        "Shimmer:DDA": float(dda) if not np.isnan(dda) else None,
        "NHR": float(nhr) if not np.isnan(nhr) else None,
        "HNR": float(hnr) if not np.isnan(hnr) else None,
        "RPDE": float(rpde) if not np.isnan(rpde) else None,
        "DFA": float(dfa) if not np.isnan(dfa) else None,
        "spread1": float(spread1) if not np.isnan(spread1) else None,
        "spread2": float(spread2) if not np.isnan(spread2) else None,
        "D2": float(d2) if not np.isnan(d2) else None,
        "PPE": float(ppe) if not np.isnan(ppe) else None,
        "status": None
    }
    return features

def save_features_to_json(features, output_file):
    with open(output_file, 'w') as f:
        json.dump(features, f, indent=4)
    print(f"Features saved to {output_file}")

# --- Train and Save RFC Model (Adapted from Your Notebook) ---

def train_and_save_rfc():
    # Load dataset
    url_string = 'https://archive.ics.uci.edu/ml/machine-learning-databases/parkinsons/parkinsons.data'
    url_content = requests.get(url_string).content
    with open('data.csv', 'wb') as data_file:
        data_file.write(url_content)
    df = pd.read_csv('data.csv')
    
    # Drop redundant column 'name'
    df = df.drop('name', axis=1)
    
    # Features and target
    X = df.drop('status', axis=1)
    y = df['status']
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=200)
    
    # Train RFC
    rfc = RandomForestClassifier(random_state=200, max_features='sqrt', n_estimators=125, max_depth=7, criterion='entropy')
    rfc.fit(X_train, y_train)
    
    # Save the model
    joblib.dump(rfc, 'rf_clf.pkl')
    print("Random Forest Classifier model saved as 'rf_clf.pkl'")
    return rfc, X.columns  # Return column names for feature order

# --- Predict Using JSON Input ---

def predict_from_json(json_file, model_file='rf_clf.pkl'):
    # Load the trained model
    rfc = joblib.load(model_file)
    print(f"Loaded Random Forest Classifier from {model_file}")
    
    # Load JSON file
    with open(json_file, 'r') as f:
        features = json.load(f)
    
    # Feature order from the training data
    feature_order = [
        "MDVP:Fo(Hz)", "MDVP:Fhi(Hz)", "MDVP:Flo(Hz)", "MDVP:Jitter(%)", "MDVP:Jitter(Abs)",
        "MDVP:RAP", "MDVP:PPQ", "Jitter:DDP", "MDVP:Shimmer", "MDVP:Shimmer(dB)",
        "Shimmer:APQ3", "Shimmer:APQ5", "MDVP:APQ", "Shimmer:DDA", "NHR", "HNR",
        "RPDE", "DFA", "spread1", "spread2", "D2", "PPE"
    ]
    
    # Prepare input array
    input_array = np.array([features[feature] if features[feature] is not None else 0 for feature in feature_order])
    input_array = input_array.reshape(1, -1)  # Shape: (1, 22)
    
    # Predict
    predicted_class = rfc.predict(input_array)
    probabilities = rfc.predict_proba(input_array)
    
    # Result
    result = {
        "predicted_class": int(predicted_class[0]),
        "probability_healthy": float(probabilities[0][0]),
        "probability_parkinsons": float(probabilities[0][1])
    }
    print(f"Predicted Class: {result['predicted_class']} (0 = Healthy, 1 = Parkinson's)")
    print(f"Probability Healthy: {result['probability_healthy']:.4f}")
    print(f"Probability Parkinson's: {result['probability_parkinsons']:.4f}")
    return result

# --- Main Execution ---

if __name__ == "__main__":
    # Paths
    audio_file = r"C:\Users\sures\OneDrive\Desktop\parkinsons-code\PD_AH\AH_545648867-CB17D873-1CEA-492A-B5B0-93C7463F516C.wav"
    json_file = r"C:\Users\sures\OneDrive\Desktop\parkinsons-code\features4.json"
    model_file = r"C:\Users\sures\OneDrive\Desktop\parkinsons-code\rf_clf.pkl"
    
    # Step 1: Extract features and save to JSON
    try:
        features = extract_voice_features(audio_file)
        save_features_to_json(features, json_file)
    except Exception as e:
        print(f"Error extracting features: {e}")
        exit()
    
    # Step 2: Train and save RFC model (run once, comment out if model already exists)
    try:
        rfc, feature_columns = train_and_save_rfc()
    except Exception as e:
        print(f"Error training RFC: {e}")
        exit()
    
    # Step 3: Predict using the JSON file
    try:
        predict_from_json(json_file, model_file)
    except Exception as e:
        print(f"Error predicting from JSON: {e}")

Features saved to C:\Users\sures\OneDrive\Desktop\parkinsons-code\features4.json
Error training RFC: name 'requests' is not defined
Loaded Random Forest Classifier from C:\Users\sures\OneDrive\Desktop\parkinsons-code\rf_clf.pkl
Predicted Class: 1 (0 = Healthy, 1 = Parkinson's)
Probability Healthy: 0.0320
Probability Parkinson's: 0.9680
